In [14]:
import math
from pickle import FALSE

import cv2
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
from numpy.ma.core import left_shift

In [15]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(
    static_image_mode=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    model_complexity = 1
)

In [ ]:
def detectPose(image, pose, display=True):
    '''
    Perform pose detection on an image.
    Args:
        image: Input image with a prominent person for pose detection.
        pose: Mediapipe pose function for detection.
        display: If True, display the images and landmarks.
    Returns:
        output_image: Image with detected pose landmarks drawn.
        landmarks: List of detected landmarks in original scale.
    '''
    # Create a copy of the input image.
    output_image = image.copy()

    # Convert BGR to RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform Pose Detection.
    results = pose.process(imageRGB)

    # Retrieve the image dimensions.
    height, width, _ = image.shape

    # Initialize list for landmarks.
    landmarks = []

    # If landmarks are detected.
    if results.pose_landmarks:
        # Draw Pose landmarks on the image.
        mp_drawing.draw_landmarks(
            image=output_image,
            landmark_list=results.pose_landmarks,
            connections=mp_pose.POSE_CONNECTIONS
        )

        # Save the landmarks.
        for landmark in results.pose_landmarks.landmark:
            landmarks.append((int(landmark.x * width), int(landmark.y * height), landmark.z * width))

    # Display results if requested.
    if display:
        plt.figure(figsize=[22, 22])
        plt.subplot(121);
        plt.imshow(image[:, :, ::-1]);
        plt.title("Original Image");
        plt.axis('off');
        plt.subplot(122);
        plt.imshow(output_image[:, :, ::-1]);
        plt.title("Output Image");
        plt.axis('off');
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    else:
        return output_image, landmarks

In [17]:

# Initialize Pose function for video.
pose_video = mp_pose.Pose(
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
    model_complexity=1
)

# Open the webcam (change index to 1 if another camera is used).
video = cv2.VideoCapture(0)

# Create named window for resizing.
cv2.namedWindow('Pose Detection', cv2.WINDOW_NORMAL)

# Set video resolution.
video.set(3, 1280)  # Width
video.set(4, 960)  # Height

# Initialize time for FPS calculation.
time1 = 0

# Video capture loop.
while video.isOpened():
    # Read a frame.
    ok, frame = video.read()

    # If frame is not read properly, break the loop.
    if not ok:
        print("Error: Unable to read frame.")
        break

    # Flip the frame horizontally for natural view.
    frame = cv2.flip(frame, 1)

    # Resize the frame.
    frame_height, frame_width, _ = frame.shape
    frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))

    # Perform pose detection.
    try:
        # Convert the frame to RGB for processing.
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform pose detection.
        results = pose_video.process(rgb_frame)

        # If landmarks are detected, process them.
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Print and optionally process landmarks.
            for i, landmark in enumerate(landmarks):
                print(
                    f"Landmark {i}: x={landmark.x:.3f}, y={landmark.y:.3f}, z={landmark.z:.3f}, visibility={landmark.visibility:.2f}"
                )

                # Optionally, convert normalized coordinates to pixels.
                pixel_x = int(landmark.x * frame_width)
                pixel_y = int(landmark.y * frame_height)
                print(f"Landmark {i} in pixels: x={pixel_x}, y={pixel_y}")

            # Draw pose landmarks on the frame.
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )

    except Exception as e:
        print("Error during pose detection:", e)
        continue

    # Calculate and display FPS.
    time2 = time()
    if (time2 - time1) > 0:
        fps = 1.0 / (time2 - time1)
        cv2.putText(frame, f'FPS: {int(fps)}', (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
    time1 = time2

    # Display the frame.
    cv2.imshow('Pose Detection', frame)

    # Break on 'ESC' key press.
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release the video capture object and close windows.
video.release()
cv2.destroyAllWindows()


Landmark 0: x=0.413, y=0.558, z=-1.072, visibility=1.00
Landmark 0 in pixels: x=528, y=401
Landmark 1: x=0.436, y=0.469, z=-1.002, visibility=1.00
Landmark 1 in pixels: x=558, y=337
Landmark 2: x=0.451, y=0.469, z=-1.002, visibility=1.00
Landmark 2 in pixels: x=576, y=337
Landmark 3: x=0.465, y=0.471, z=-1.002, visibility=1.00
Landmark 3 in pixels: x=595, y=338
Landmark 4: x=0.386, y=0.464, z=-1.011, visibility=1.00
Landmark 4 in pixels: x=494, y=334
Landmark 5: x=0.366, y=0.461, z=-1.010, visibility=1.00
Landmark 5 in pixels: x=468, y=332
Landmark 6: x=0.348, y=0.459, z=-1.011, visibility=1.00
Landmark 6 in pixels: x=445, y=330
Landmark 7: x=0.482, y=0.499, z=-0.508, visibility=1.00
Landmark 7 in pixels: x=617, y=359
Landmark 8: x=0.311, y=0.478, z=-0.524, visibility=1.00
Landmark 8 in pixels: x=397, y=344
Landmark 9: x=0.437, y=0.640, z=-0.878, visibility=1.00
Landmark 9 in pixels: x=559, y=460
Landmark 10: x=0.381, y=0.636, z=-0.885, visibility=1.00
Landmark 10 in pixels: x=487, y=4

KeyboardInterrupt: 

In [18]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX


In [19]:
landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]


x: -0.0298345555
y: 1.28193176
z: -0.808583915
visibility: 0.330686361

In [20]:
#lets see this one
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Midpoint
    c = np.array(c)  # End point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle 

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Extract relevant points
            joint_sets = {
                "Left Arm": [mp_pose.PoseLandmark.LEFT_SHOULDER, 
                             mp_pose.PoseLandmark.LEFT_ELBOW, 
                             mp_pose.PoseLandmark.LEFT_WRIST],
                
                "Right Arm": [mp_pose.PoseLandmark.RIGHT_SHOULDER, 
                              mp_pose.PoseLandmark.RIGHT_ELBOW, 
                              mp_pose.PoseLandmark.RIGHT_WRIST],

                "Left Leg": [mp_pose.PoseLandmark.LEFT_HIP, 
                             mp_pose.PoseLandmark.LEFT_KNEE, 
                             mp_pose.PoseLandmark.LEFT_ANKLE],

                "Right Leg": [mp_pose.PoseLandmark.RIGHT_HIP, 
                              mp_pose.PoseLandmark.RIGHT_KNEE, 
                              mp_pose.PoseLandmark.RIGHT_ANKLE],

                "Torso": [mp_pose.PoseLandmark.LEFT_SHOULDER, 
                          mp_pose.PoseLandmark.LEFT_HIP, 
                          mp_pose.PoseLandmark.LEFT_KNEE]
            }

            for joint_name, points in joint_sets.items():
                p1 = [landmarks[points[0].value].x, landmarks[points[0].value].y]
                p2 = [landmarks[points[1].value].x, landmarks[points[1].value].y]
                p3 = [landmarks[points[2].value].x, landmarks[points[2].value].y]
                
                angle = calculate_angle(p1, p2, p3)
                
                # Print angle values to console
                print(f"{joint_name} angle: {angle:.2f}")

                # Visualize angle on screen
                cv2.putText(image, f"{joint_name}: {int(angle)}", 
                            tuple(np.multiply(p2, [640, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        except:
            pass
        
        # Render pose landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe Pose Estimation', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Left Arm angle: 157.01
Right Arm angle: 164.51
Left Leg angle: 179.50
Right Leg angle: 179.38
Torso angle: 175.99
Left Arm angle: 148.40
Right Arm angle: 165.88
Left Leg angle: 178.82
Right Leg angle: 178.99
Torso angle: 175.55
Left Arm angle: 146.16
Right Arm angle: 166.18
Left Leg angle: 178.93
Right Leg angle: 178.95
Torso angle: 175.73
Left Arm angle: 147.19
Right Arm angle: 166.21
Left Leg angle: 178.52
Right Leg angle: 178.67
Torso angle: 175.17
Left Arm angle: 145.53
Right Arm angle: 166.97
Left Leg angle: 178.70
Right Leg angle: 178.70
Torso angle: 175.71
Left Arm angle: 141.12
Right Arm angle: 167.06
Left Leg angle: 178.74
Right Leg angle: 178.90
Torso angle: 176.17
Left Arm angle: 142.50
Right Arm angle: 167.31
Left Leg angle: 178.71
Right Leg angle: 178.78
Torso angle: 176.34
Left Arm angle: 136.79
Right Arm angle: 167.45
Left Leg angle: 178.50
Right Leg angle: 178.61
Torso angle: 176.70
Left Arm angle: 139.88
Right Arm angle: 167.49
Left Leg angle: 178.67
Right Leg angle: 1

KeyboardInterrupt: 

In [7]:
#################### Pose Identify Using Values
def calculateAngle(landmark1, landmark2, landmark3):
    '''
    This function calculates angle between three different landmarks.
    Args:
        landmark1: The first landmark containing the x,y and z coordinates.
        landmark2: The second landmark containing the x,y and z coordinates.
        landmark3: The third landmark containing the x,y and z coordinates.
    Returns:
        angle: The calculated angle between the three landmarks.

    '''

    # Get the required landmarks coordinates.
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    # Check if the angle is less than zero.
    if angle < 0:

        # Add 360 to the found angle.
        angle += 360
    
    # Return the calculated angle.
    return angle

In [8]:
# Calculate the angle between the three landmarks.
angle = calculateAngle((558, 326, 0), (642, 333, 0), (718, 321, 0))

# Display the calculated angle.
print(f'The calculated angle is {angle}')

The calculated angle is 166.26373169437744


In [10]:
def classifyPose(landmarks, output_image, display=False):
    '''
    This function classifies yoga poses depending upon the angles of various body joints.
    Args:
        landmarks: A list of detected landmarks of the person whose pose needs to be classified.
        output_image: A image of the person with the detected pose landmarks drawn.
        display: A boolean value that is if set to true the function displays the resultant image with the pose label 
        written on it and returns nothing.
    Returns:
        output_image: The image with the detected pose landmarks drawn and pose label written.
        label: The classified pose label of the person in the output_image.

    '''
    
    # Initialize the label of the pose. It is not known at this stage.
    label = 'Unknown Pose'

    # Specify the color (Red) with which the label will be written on the image.
    color = (0, 0, 255)
    
    # Calculate the required angles.
    #----------------------------------------------------------------------------------------------------------------
    
    # Get the angle between the left shoulder, elbow and wrist points. 
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    
    # Get the angle between the right shoulder, elbow and wrist points. 
    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])   
    
    # Get the angle between the left elbow, shoulder and hip points. 
    left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])

    # Get the angle between the right hip, shoulder and elbow points. 
    right_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])

    # Get the angle between the left hip, knee and ankle points. 
    left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])

    # Get the angle between the right hip, knee and ankle points 
    right_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if it is the warrior II pose or the T pose.
    # As for both of them, both arms should be straight and shoulders should be at the specific angle.
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if the both arms are straight.
    if left_elbow_angle > 165 and left_elbow_angle < 195 and right_elbow_angle > 165 and right_elbow_angle < 195:

        # Check if shoulders are at the required angle.
        if left_shoulder_angle > 80 and left_shoulder_angle < 110 and right_shoulder_angle > 80 and right_shoulder_angle < 110:

    # Check if it is the warrior II pose.
    #----------------------------------------------------------------------------------------------------------------

            # Check if one leg is straight.
            if left_knee_angle > 165 and left_knee_angle < 195 or right_knee_angle > 165 and right_knee_angle < 195:

                # Check if the other leg is bended at the required angle.
                if left_knee_angle > 90 and left_knee_angle < 120 or right_knee_angle > 90 and right_knee_angle < 120:

                    # Specify the label of the pose that is Warrior II pose.
                    label = 'Warrior II Pose' 
                        
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if it is the T pose.
    #----------------------------------------------------------------------------------------------------------------
    
            # Check if both legs are straight
            if left_knee_angle > 160 and left_knee_angle < 195 and right_knee_angle > 160 and right_knee_angle < 195:

                # Specify the label of the pose that is tree pose.
                label = 'T Pose'

    #----------------------------------------------------------------------------------------------------------------
    
    # Check if it is the tree pose.
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if one leg is straight
    if left_knee_angle > 165 and left_knee_angle < 195 or right_knee_angle > 165 and right_knee_angle < 195:

        # Check if the other leg is bended at the required angle.
        if left_knee_angle > 315 and left_knee_angle < 335 or right_knee_angle > 25 and right_knee_angle < 45:

            # Specify the label of the pose that is tree pose.
            label = 'Tree Pose'
                
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if the pose is classified successfully
    if label != 'Unknown Pose':
        
        # Update the color (to green) with which the label will be written on the image.
        color = (0, 255, 0)  
    
    # Write the label on the output image. 
    cv2.putText(output_image, label, (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
    
    # Check if the resultant image is specified to be displayed.
    if display:
    
        # Display the resultant image.
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
    else:
        
        # Return the output image and the classified label.
        return output_image, label